<a href="https://colab.research.google.com/github/AhnYeonghoo/Plastic-self-classification-system-using-deep-learning/blob/Machine-Learning/Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import json
import cv2
import shutil
from sklearn.model_selection import train_test_split

# Step 1: YOLOv7 저장소 클론
if not os.path.exists('yolov7'):
    !git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7
!pip install -r requirements.txt

# Step 2: JSON 어노테이션을 YOLO 형식으로 변환
def convert_bbox(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = box[0] + box[2] / 2.0
    y = box[1] + box[3] / 2.0
    w = box[2]
    h = box[3]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def convert_annotations(json_dir, output_dir, image_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    json_files = [f for f in os.listdir(json_dir) if f.endswith('.json')]

    class_names = ['c_5_02', 'c_6', 'c_5_01_01', 'c_5_02_01', 'c_6_01']
    class_id_map = {name: i for i, name in enumerate(class_names)}

    for json_file in json_files:
        with open(os.path.join(json_dir, json_file)) as f:
            data = json.load(f)

        image_path = os.path.join(image_dir, data['Image'])

        image = cv2.imread(image_path)
        if image is None:
            print(f"이미지를 찾을 수 없습니다: {image_path}")
            continue

        h, w = 720, 1280  # 제공된 이미지 해상도

        with open(os.path.join(output_dir, json_file.replace('.json', '.txt')), 'w') as out_file:
            for obj in data['objects']:
                class_name = obj['class_name']
                if class_name not in class_id_map:
                    continue
                class_id = class_id_map[class_name]
                coord = obj['annotation']['coord']
                box = (coord['x'], coord['y'], coord['width'], coord['height'])
                bbox = convert_bbox((w, h), box)
                out_file.write(f"{class_id} {' '.join(map(str, bbox))}\n")

json_dir = '/content/drive/MyDrive/Sample/02.라벨링데이터'
output_dir = '/content/yolov7/labels'
image_dir = '/content/drive/MyDrive/Sample/01.원천데이터'
convert_annotations(json_dir, output_dir, image_dir)

# Step 3: 데이터셋 디렉토리 구조 설정 및 데이터 분할
!mkdir -p /content/yolov7/data/images/train
!mkdir -p /content/yolov7/data/images/val
!mkdir -p /content/yolov7/data/labels/train
!mkdir -p /content/yolov7/data/labels/val

# 모든 이미지를 가져와서 학습 및 평가용으로 분할
all_images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
train_images, val_images = train_test_split(all_images, test_size=0.2, random_state=42)

# 이미지와 라벨 파일 이동
def move_files(image_list, destination_image_dir, destination_label_dir):
    for image_name in image_list:
        label_name = image_name.replace('.jpg', '.txt')
        shutil.copy(os.path.join(image_dir, image_name), destination_image_dir)
        if os.path.exists(os.path.join(output_dir, label_name)):
            shutil.copy(os.path.join(output_dir, label_name), destination_label_dir)

move_files(train_images, '/content/yolov7/data/images/train', '/content/yolov7/data/labels/train')
move_files(val_images, '/content/yolov7/data/images/val', '/content/yolov7/data/labels/val')

# Step 4: dataset.yaml 생성
dataset_yaml = """
train: /content/yolov7/data/images/train
val: /content/yolov7/data/images/val

nc: 5
names: ['c_5_02', 'c_6', 'c_5_01_01', 'c_5_02_01', 'c_6_01']
"""

with open('/content/yolov7/data/dataset.yaml', 'w') as f:
    f.write(dataset_yaml)

# Step 5: 모델 학습
!python train.py --img 640 --batch 16 --epochs 100 --data /content/yolov7/data/dataset.yaml --cfg cfg/training/yolov7.yaml --weights 'yolov7.pt' --name yolov7_custom

# Step 6: 모델 평가
!python test.py --data /content/yolov7/data/dataset.yaml --weights runs/train/yolov7_custom/weights/best.pt --img 640 --conf 0.001 --iou 0.65 --task test --device 0

# Step 7: mAP 및 기타 지표 출력
import glob

def display_metrics():
    results_files = glob.glob('runs/test/exp*/results*.txt')
    if results_files:
        with open(results_files[0], 'r') as file:
            results = file.read()
            print(results)

display_metrics()
